In [1]:
import pandas as pd
import nltk 
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import pickle      

In [3]:
df_data = pickle.load(open('5.PM_MANMOHAN_Speech_Cleaned_string.pickle', 'rb'))

In [5]:
df_data

,index,Date,Location,Title,Speech,Cleaned_Speech,Cleaned_Speech_string
0,MSS1,2004-06-24,New Delhi,Prime Minister 's Address to the Nation,"My Fellow Citizens,I speak to you with a deep...",['fellow citizen speak deep sense humility ful...,fellow citizen speak deep sense humility fully...
1,MSS2,2004-06-29,New Delhi,PM's inaugural address at the Chief Ministers'...,"""I welcome you all to this Conference which w...",['welcome conference take stock effort rural d...,welcome conference take stock effort rural dev...
2,MSS3,2004-07-03,New Delhi,"PM's inaugural address at the ""Dialogue on Min...","""It gives me great pleasure to be amongst you...","['give great pleasure amongst morning', 'parti...",give great pleasure amongst morning particular...
3,MSS4,2004-07-26,New Delhi,PM's speech at the CSIR Society Meeting,"Distinguished Members of the CSIR Society, La...",['distinguished member csir society lady gentl...,distinguished member csir society lady gentlem...
4,MSS5,2004-07-31,Bangkok,PM’ statement at the Press Conference,"\n\t""My two days in Bangkok have been very use...","['two day bangkok useful productive', 'first b...",two day bangkok useful productive first bimste...
...,...,...,...,...,...,...,...
1346,MSS1397,2014-02-21,New Delhi,PM’s farewell speech to the 15th Lok Sabha,"""Madam Speaker,\n\nAs we come to the end of th...",['madam speaker come end journey fifteenth lok...,madam speaker come end journey fifteenth lok s...
1347,MSS1398,2014-02-26,New Delhi,PM's speech at the foundation laying ceremony ...,\n\t“Today marks a very important step forward...,['today mark important step forward towards es...,today mark important step forward towards esta...
1348,MSS1399,2014-03-01,New Delhi,PM’s address at the National Awards Ceremony f...,\n\t“I am very happy to participate in the Nat...,['happy participate national award function mi...,happy participate national award function mini...
1349,MSS1400,2014-03-04,"Nay Pyi Taw, Myanmar",PM’s statement at 3rd BIMSTEC Summit,\n\t“I am delighted to return to this beautifu...,['delighted return beautiful city nay pyi taw ...,delighted return beautiful city nay pyi taw th...


In [6]:
def model(corpus):
    corpus_tokenized = [nltk.word_tokenize(sentence) for sentence in corpus]
    model = Word2Vec(sentences=corpus_tokenized, vector_size=300, window=5, min_count=1, workers=4)
    return model 


In [7]:
# Defining the function for vectorization 
def predict(model, speech):
    vectors = []

    for sentence in speech:
        sent_vec = []
        for word in sentence:
            if word in model.wv.key_to_index:
                # If the word is in the Word2Vec model's vocabulary, extract its vector
                word_vector = model.wv.get_vector(word)
                sent_vec.append(word_vector)
        vectors.append(sent_vec)

    return vectors 



In [8]:
# Defining a function to count the number of unique words in each sentences 
def count_unique_words(j):
    m = j.replace('[','').replace(']','').replace("'","").split(', ')
    l = []
    for k in m:
        l += k.split(" ")
    length = len(l)
    return length 

In [9]:
# Creating the corpus of all the speeches to train the model 
corpus_Manmohan = []
for i in df_data['Cleaned_Speech']:
    corpus_Manmohan += i.replace('[','').replace(']','').replace("'","").split(', ')

In [21]:
empty_count = 0
for element in corpus_Manmohan:
    if element.strip() == '':
        empty_count += 1

print(f'Total empty elements in the list: {empty_count}')

Total empty elements in the list: 0


In [11]:
len(corpus_Manmohan)

76646

In [12]:
corpus_Manmohan = [element for element in corpus_Manmohan if element.strip() != '']

In [13]:
len(corpus_Manmohan)

76590

In [14]:
# Training the model 
model_manmohan = model(corpus_Manmohan)

In [15]:
Word2vec_model_MM = open("Word2vec_model_MM.pickle", "wb")
pickle.dump(model_manmohan, Word2vec_model_MM)
Word2vec_model_MM.close()

In [22]:
# Creating the list containing each of the speeches 
speech_manmohan = []
for i in df_data['Cleaned_Speech']:
    speech_manmohan.append(i.replace('[','').replace(']','').replace("'","").split(', '))

In [28]:
count = 0
for sp in speech_manmohan:
    for sent in sp:
        if len(sent)==0:
            count +=1

print(count)

0


In [26]:
len(speech_manmohan)

1351

In [27]:
speech_manmohan = [[sent for sent in sp if len(sent) > 0] for sp in speech_manmohan]

In [29]:
# Vectorize each speech 
speech_manmohan_vector = []
for sp in speech_manmohan:
    speech_manmohan_vector.append(predict(model_manmohan, sp)) 

In [30]:
import pandas as pd
import numpy as np

# initialize an empty list to store the row dictionaries
rows = []

# loop over all speeches, sentences, and words
for speech_number, speech in enumerate(speech_manmohan_vector):
    for sentence_number, sentence in enumerate(speech):
        # initialize a vector of zeros with 300 dimensions
        sentence_vector = np.zeros(300)
        for word in sentence:
            # add the ith element of the word vector to the corresponding dimension of the sentence vector
            sentence_vector += np.array(word)
        # divide the sentence vector by the number of words to get the average
        sentence_vector /= len(sentence)
        # create a dictionary with the speech number, sentence number, and sentence vector
        row_dict = {"speech_number": speech_number, "sentence_number": sentence_number}
        for i, dim in enumerate(sentence_vector):
            row_dict["dim_" + str(i)] = dim
        # add the dictionary to the list of rows
        rows.append(row_dict)

# create a dataframe from the list of rows
df = pd.DataFrame(rows)


In [31]:
df

,speech_number,sentence_number,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
0,0,0,0.039272,0.141355,0.208785,0.069060,0.104892,-0.260684,-0.116273,0.375820,...,0.011337,0.204659,0.095231,-0.086294,-0.043487,0.141908,-0.058881,-0.082777,0.239641,-0.118006
1,0,1,0.120887,0.130800,0.338174,0.093093,0.224007,-0.347564,-0.185856,0.509600,...,-0.034526,0.261834,0.090970,-0.084383,-0.030871,0.195252,-0.125502,-0.021038,0.348104,-0.174912
2,0,2,0.068629,0.116968,0.313716,0.046078,0.220565,-0.362391,-0.234886,0.587728,...,-0.094740,0.253656,0.018483,-0.177701,0.017921,0.148291,-0.076770,-0.016883,0.397155,-0.094534
3,0,3,0.141265,0.142769,0.378398,0.099277,0.244987,-0.377983,-0.199397,0.542631,...,-0.028668,0.292756,0.105268,-0.079638,-0.044509,0.218023,-0.146637,-0.023908,0.376462,-0.198925
4,0,4,0.075832,0.142868,0.257141,0.097784,0.114247,-0.300039,-0.141870,0.414136,...,0.016309,0.224906,0.100370,-0.082233,-0.052099,0.170813,-0.085070,-0.081079,0.275172,-0.145259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76585,1350,18,0.134553,0.158850,0.298368,0.141190,0.111383,-0.286182,-0.102686,0.339379,...,0.089979,0.221411,0.161208,0.026411,-0.127948,0.223995,-0.146779,-0.088264,0.235623,-0.236756
76586,1350,19,0.102642,0.133537,0.315415,0.098128,0.185659,-0.346090,-0.197507,0.503516,...,-0.037271,0.252121,0.075691,-0.111676,-0.022353,0.181521,-0.102612,-0.035719,0.345514,-0.154237
76587,1350,20,0.137535,0.210427,0.502807,0.216045,0.070138,-0.439591,-0.289210,0.517369,...,0.139286,0.227958,0.137341,-0.032523,-0.258146,0.335559,-0.188988,-0.164049,0.400045,-0.327823
76588,1350,21,0.068292,0.108950,0.199079,0.110282,0.038485,-0.215576,-0.103870,0.268292,...,0.065710,0.118497,0.082431,-0.010440,-0.078957,0.163896,-0.088141,-0.090542,0.173805,-0.151080


In [32]:
# group the rows by speech number and compute the mean of each group
df_speech = df.groupby("speech_number").mean()

# reset the index of the dataframe and rename the columns
df_speech = df_speech.reset_index().rename(columns={"index": "speech_number"})

# re-order the columns of the dataframe
df_speech = df_speech[["speech_number"] + ["dim_" + str(i) for i in range(300)]]


In [33]:
df_speech.insert(0, "index", df_data["index"])
df_speech.insert(1, "Date", df_data["Date"])

In [35]:
print(df_data.shape, df_speech.shape)

(1351, 7) (1351, 303)


In [38]:
df_speech

,index,Date,speech_number,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
0,MSS1,2004-06-24,1,0.080297,0.134279,0.284168,0.076520,0.179103,-0.322670,-0.170644,...,-0.031048,0.242218,0.078113,-0.105497,-0.018716,0.169930,-0.090247,-0.043599,0.319455,-0.136552
1,MSS2,2004-06-29,2,0.079093,0.134818,0.281025,0.077084,0.177635,-0.321879,-0.168710,...,-0.029820,0.241174,0.078981,-0.104977,-0.017535,0.169701,-0.089239,-0.044774,0.317339,-0.136240
2,MSS3,2004-07-03,3,0.069196,0.136458,0.258998,0.073378,0.163749,-0.305695,-0.151336,...,-0.022496,0.235061,0.083821,-0.100795,-0.017868,0.161450,-0.081704,-0.051715,0.295892,-0.128903
3,MSS4,2004-07-26,4,0.093168,0.123642,0.278399,0.076835,0.200466,-0.332305,-0.158284,...,-0.028054,0.248509,0.082217,-0.089680,0.005598,0.169920,-0.094163,-0.029768,0.317748,-0.135326
4,MSS5,2004-07-31,5,0.117241,0.102200,0.300157,0.056820,0.283920,-0.365327,-0.152803,...,-0.035231,0.279282,0.088477,-0.067186,0.035104,0.168210,-0.103989,0.003956,0.352150,-0.127860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346,MSS1397,2014-02-21,1347,0.076581,0.137236,0.280551,0.073935,0.181614,-0.319405,-0.162057,...,-0.026160,0.241607,0.080991,-0.100646,-0.022199,0.169359,-0.091316,-0.045952,0.312743,-0.138842
1347,MSS1398,2014-02-26,1348,0.084719,0.125890,0.294288,0.068374,0.203356,-0.338564,-0.174935,...,-0.032406,0.250723,0.073761,-0.102107,-0.007536,0.170501,-0.091737,-0.034616,0.335924,-0.131072
1348,MSS1399,2014-03-01,1349,0.082907,0.129979,0.291531,0.075198,0.192613,-0.337205,-0.178759,...,-0.036618,0.248607,0.074745,-0.110865,-0.007053,0.172062,-0.089604,-0.038163,0.334069,-0.131994
1349,MSS1400,2014-03-04,1350,0.089641,0.129296,0.296178,0.071581,0.206227,-0.329946,-0.172299,...,-0.040815,0.250162,0.074646,-0.101385,-0.010126,0.172244,-0.098906,-0.028418,0.329811,-0.139486


In [36]:
df_speech['speech_number'] = df_speech['speech_number'] +1

In [39]:
word2vec_MM = open("word2vec_MM.pickle", "wb")
pickle.dump(df_speech, word2vec_MM)
word2vec_MM.close()